<a href="https://colab.research.google.com/github/chaotingchong-crypto/programming-language/blob/main/HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install gspread gspread_dataframe google-auth google-auth-oauthlib google-auth-httplib2 \
               gradio pandas beautifulsoup4 google-generativeai python-dateutil plotly


In [2]:
import os, time, uuid, re, json, datetime
from datetime import datetime as dt, timedelta
from dateutil.tz import gettz
import pandas as pd
import gradio as gr
import requests
from bs4 import BeautifulSoup

import google.generativeai as genai

# Google Auth & Sheets
from google.colab import auth
import gspread
from gspread_dataframe import set_with_dataframe, get_as_dataframe
from google.auth.transport.requests import Request
from google.oauth2 import service_account
from google.auth import default


In [3]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [4]:
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Google Sheet 設定
SHEET_URL = "https://docs.google.com/spreadsheets/d/1yn7Qi7piNvLgnriw3lnYcQ5O_eSGlL6zeSlyvUXWVL4/edit?gid=0#gid=0"
WORKSHEET_NAME = "健身紀錄"
TIMEZONE = "Asia/Taipei"

In [5]:
def tznow():
    return dt.now(gettz(TIMEZONE))

In [6]:
# =========================
# 4️⃣ 表格欄位定義
# =========================
TASKS_HEADER = [
    "id","exercise","status","intensity","est_min","start_time","end_time",
    "actual_min","sets","training_date","category","notes",
    "created_at","updated_at","completed_at","planned_for"
]
LOGS_HEADER = [
    "log_id","exercise_id","phase","start_ts","end_ts","minutes","sets","note"
]
CLIPS_HEADER = ["clip_id","url","selector","text","href","created_at","added_to_exercise"]


In [7]:
# =========================
# 5️⃣ 建立 / 載入工作表
# =========================
def ensure_spreadsheet(name):
    try:
        sh = gc.open(name)
    except gspread.SpreadsheetNotFound:
        sh = gc.create(name)
    return sh

def ensure_worksheet(sh, title, header):
    try:
        ws = sh.worksheet(title)
    except gspread.WorksheetNotFound:
        ws = sh.add_worksheet(title=title, rows="1000", cols=str(len(header)+5))
        ws.update([header])
    data = ws.get_all_values()
    if not data or (data and data[0] != header):
        ws.clear()
        ws.update([header])
    return ws

sh = ensure_spreadsheet(WORKSHEET_NAME)
ws_tasks = ensure_worksheet(sh, "exercises", TASKS_HEADER)
ws_logs  = ensure_worksheet(sh, "training_logs", LOGS_HEADER)
ws_clips = ensure_worksheet(sh, "fitness_clips", CLIPS_HEADER)


In [8]:
# =========================
# 6️⃣ 讀寫 DataFrame 函數
# =========================
def read_df(ws, header):
    df = get_as_dataframe(ws, evaluate_formulas=True, header=0)
    if df is None or df.empty:
        return pd.DataFrame(columns=header)
    df = df.fillna("")
    for c in header:
        if c not in df.columns:
            df[c] = ""
    # 型別轉換
    for col in ["est_min","actual_min","sets"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0).astype(int)
    return df[header]

def write_df(ws, df, header):
    if df.empty:
        ws.clear()
        ws.update([header])
        return
    df_out = df.copy()
    for c in df_out.columns:
        df_out[c] = df_out[c].astype(str)
    ws.clear()
    ws.update([header] + df_out.values.tolist())

def refresh_all():
    global tasks_df, logs_df, clips_df
    tasks_df = read_df(ws_tasks, TASKS_HEADER)
    logs_df  = read_df(ws_logs, LOGS_HEADER)
    clips_df = read_df(ws_clips, CLIPS_HEADER)
    return tasks_df, logs_df, clips_df

tasks_df, logs_df, clips_df = refresh_all()


In [9]:
# =========================
# 7️⃣ 新增 / 更新 / 刪除任務
# =========================
def add_exercise(exercise, intensity, est_min, training_date, category, notes, planned_for):
    global tasks_df
    _now = tznow().isoformat()
    new = pd.DataFrame([{
        "id": str(uuid.uuid4())[:8],
        "exercise": exercise.strip(),
        "status": "未完成",
        "intensity": intensity or "中",
        "est_min": int(est_min) if est_min else 30,
        "start_time": "",
        "end_time": "",
        "actual_min": 0,
        "sets": 0,
        "training_date": training_date or "",
        "category": category or "",
        "notes": notes or "",
        "created_at": _now,
        "updated_at": _now,
        "completed_at": "",
        "planned_for": planned_for or ""
    }])
    tasks_df = pd.concat([tasks_df, new], ignore_index=True)
    write_df(ws_tasks, tasks_df, TASKS_HEADER)
    return "✅ 已新增運動項目", tasks_df

def update_exercise_status(ex_id, new_status):
    global tasks_df
    idx = tasks_df.index[tasks_df["id"]==ex_id]
    if len(idx)==0:
        return "⚠️ 找不到運動項目", tasks_df
    i = idx[0]
    tasks_df.loc[i, "status"] = new_status
    tasks_df.loc[i, "updated_at"] = tznow().isoformat()
    if new_status=="已完成" and not tasks_df.loc[i,"completed_at"]:
        tasks_df.loc[i,"completed_at"] = tznow().isoformat()
    write_df(ws_tasks, tasks_df, TASKS_HEADER)
    return "✅ 狀態已更新", tasks_df

def delete_exercise(ex_id):
    global tasks_df
    if ex_id not in tasks_df["id"].values:
        return "⚠️ 找不到運動項目", tasks_df
    tasks_df = tasks_df[tasks_df["id"]!=ex_id]
    write_df(ws_tasks, tasks_df, TASKS_HEADER)
    return "🗑️ 已刪除運動項目", tasks_df


In [10]:
# =========================
# 8️⃣ 番茄鐘模式 → 訓練紀錄
# =========================
_active_sessions = {}

def start_training(ex_id, phase, sets):
    if not ex_id: return "⚠️ 請選擇運動項目"
    _active_sessions[ex_id] = {
        "phase": phase,
        "start_ts": tznow().isoformat(),
        "sets": int(sets) if sets else 1
    }
    return f"▶️ 開始 {phase}（運動: {ex_id}）"

def end_training(ex_id, note):
    global logs_df, tasks_df
    if ex_id not in _active_sessions:
        return "⚠️ 尚未開始任何訓練"
    sess = _active_sessions.pop(ex_id)
    start = pd.to_datetime(sess["start_ts"])
    end = tznow()
    minutes = round((end - start).total_seconds()/60.0,2)
    log = pd.DataFrame([{
        "log_id": str(uuid.uuid4())[:8],
        "exercise_id": ex_id,
        "phase": sess["phase"],
        "start_ts": start.isoformat(),
        "end_ts": end.isoformat(),
        "minutes": minutes,
        "sets": int(sess["sets"]),
        "note": note or ""
    }])
    logs_df = pd.concat([logs_df, log], ignore_index=True)
    write_df(ws_logs, logs_df, LOGS_HEADER)
    # 回填實際時間
    idx = tasks_df.index[tasks_df["id"]==ex_id]
    if len(idx):
        i = idx[0]
        tasks_df.loc[i,"actual_min"] = tasks_df.loc[i,"actual_min"] + int(minutes)
        tasks_df.loc[i,"sets"] = tasks_df.loc[i,"sets"] + int(sess["sets"])
        tasks_df.loc[i,"updated_at"] = tznow().isoformat()
        write_df(ws_tasks, tasks_df, TASKS_HEADER)
    return f"⏹️ 結束 {sess['phase']}，紀錄 {minutes} 分鐘"


In [11]:
# =========================
# 9️⃣ 匯出 CSV / JSON
# =========================
def export_records():
    tasks_df.to_csv("exercises.csv", index=False)
    tasks_df.to_json("exercises.json", orient="records", force_ascii=False)
    logs_df.to_csv("training_logs.csv", index=False)
    logs_df.to_json("training_logs.json", orient="records", force_ascii=False)
    clips_df.to_csv("fitness_clips.csv", index=False)
    clips_df.to_json("fitness_clips.json", orient="records", force_ascii=False)
    return "✅ 匯出完成（可上傳到 GitHub）"


In [12]:
# =========================
# 🔟 今日統計 / 視覺化
# =========================
def today_summary():
    today = tznow().date().isoformat()
    planned = tasks_df[(tasks_df["training_date"]==today) | (tasks_df["planned_for"].str.lower()=="today")]
    done = planned[planned["status"]=="已完成"]
    total = len(planned)
    done_n = len(done)
    rate = (done_n/total*100) if total>0 else 0
    return f"📅 今日運動總數：{total}；✅ 完成：{done_n}；📈 完成率：{rate:.1f}%"

def plot_training_distribution():
    if tasks_df.empty: return None
    fig = px.bar(tasks_df, x="exercise", y="sets", color="status", title="各運動組數分布")
    return fig


In [14]:
# =========================
# Gradio UI
# =========================
with gr.Blocks(title="健身紀錄管理系統") as demo:
    gr.Markdown("# 🏋️ 健身紀錄系統（CSV 模擬 Google Sheet）")

    with gr.Row():
        btn_refresh = gr.Button("🔄 重新整理")
        out_summary = gr.Markdown(today_summary())

    # ==== 健身計畫分頁 ====
    with gr.Tab("健身計畫"):
        with gr.Row():
            with gr.Column(scale=2):
                exercise = gr.Textbox(label="運動項目", placeholder="深蹲 / 跑步 / 瑜伽")
                intensity = gr.Dropdown(["高","中","低"], value="中", label="強度")
                est_min = gr.Number(value=30, label="預計分鐘", precision=0)
                training_date = gr.Textbox(label="訓練日期 YYYY-MM-DD")
                category = gr.Textbox(label="分類（有氧 / 重量 / 核心）")
                notes = gr.Textbox(label="備註")
                planned_for = gr.Dropdown(["","today","tomorrow"], value="", label="規劃")
                btn_add = gr.Button("➕ 新增運動項目")
                msg_add = gr.Markdown()
            with gr.Column(scale=3):
                grid_tasks = gr.Dataframe(value=tasks_df, label="運動清單", interactive=False)

        with gr.Row():
            exercise_choice = gr.Dropdown(choices=list_exercise_choices(), label="選取運動項目")
            new_status = gr.Dropdown(["未完成","進行中","已完成"], value="進行中", label="更新狀態")
            btn_update = gr.Button("✏️ 更新狀態")
            btn_delete = gr.Button("🗑️ 刪除運動項目")
            msg_update = gr.Markdown()

    # ==== 訓練紀錄分頁 ====
    with gr.Tab("訓練紀錄"):
        sel_exercise = gr.Dropdown(choices=list_exercise_choices(), label="選擇運動")
        sets = gr.Number(value=1, precision=0, label="組數")
        btn_start = gr.Button("▶️ 開始訓練")
        note = gr.Textbox(label="備註")
        btn_end = gr.Button("⏹️ 結束訓練")
        msg_train = gr.Markdown()
        grid_logs = gr.Dataframe(value=logs_df, label="訓練紀錄", interactive=False)

    # ==== 今日統計 ====
    with gr.Tab("今日統計"):
        btn_summary = gr.Button("📊 更新今日統計")
        out_summary2 = gr.Markdown()
        fig_plot = gr.Plot()

    # ==== 綁定按鈕 ====
    btn_refresh.click(refresh_all, outputs=[grid_tasks, grid_logs, exercise_choice, out_summary])
    btn_add.click(add_exercise, inputs=[exercise,intensity,est_min,training_date,category,notes,planned_for], outputs=[msg_add, grid_tasks])
    btn_update.click(update_exercise_status, inputs=[exercise_choice,new_status], outputs=[msg_update, grid_tasks])
    btn_delete.click(delete_exercise, inputs=[exercise_choice], outputs=[msg_update, grid_tasks])
    btn_start.click(start_training, inputs=[sel_exercise, gr.State("work"), sets], outputs=[msg_train])
    btn_end.click(end_training, inputs=[sel_exercise,note], outputs=[msg_train])
    btn_summary.click(today_summary, outputs=[out_summary2])
    btn_summary.click(plot_training_distribution, outputs=[fig_plot])

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d21140f5d9b8b85146.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
